In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn import cluster
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,mutual_info_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
import time
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.tree import DecisionTreeClassifier
import inspect, re
from sklearn.preprocessing import PolynomialFeatures
from itertools import product
from sklearn.preprocessing import OrdinalEncoder
from tqdm import tqdm
import gc
from sklearn.feature_selection import VarianceThreshold,r_regression,SelectKBest,f_regression,SelectPercentile,chi2,f_classif,mutual_info_regression,mutual_info_classif,RFE,RFECV,SequentialFeatureSelector,SelectFromModel
from scipy import stats
import scipy
import random
from scipy.special import digamma
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,RepeatedKFold,KFold

In [30]:
data=pd.read_csv('newdata.csv')
# data = data.drop(columns=['roles'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11722 entries, 0 to 11721
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   course_id          11722 non-null  object 
 1   userid_DI          11722 non-null  object 
 2   registered         11722 non-null  int64  
 3   viewed             11722 non-null  int64  
 4   explored           11722 non-null  int64  
 5   certified          11722 non-null  int64  
 6   final_cc_cname_DI  11722 non-null  object 
 7   LoE_DI             11722 non-null  object 
 8   YoB                11722 non-null  int64  
 9   gender             11722 non-null  object 
 10  grade              11722 non-null  int64  
 11  start_time_DI      11722 non-null  object 
 12  last_event_DI      11722 non-null  object 
 13  nevents            11722 non-null  int64  
 14  ndays_act          11722 non-null  int64  
 15  nplay_video        6032 non-null   float64
 16  nchapters          117

In [31]:
# 现在我们将查看每个标签的数据量
label_counts = data['grade'].value_counts()
label_counts

1    6049
0    5673
Name: grade, dtype: int64

In [32]:
# 现在我们将查看每个标签的数据量
counts = data['certified'].value_counts()
counts

0    6000
1    5722
Name: certified, dtype: int64

In [36]:
def cate_colName(Transformer, category_cols, drop='if_binary'):
    """
    离散字段独热编码后字段名创建函数
    
    :param Transformer: 独热编码转化器
    :param category_cols: 输入转化器的离散变量
    :param drop: 独热编码转化器的drop参数
    """
    
    cate_cols_new = []
    col_value = Transformer.categories_
    
    for i, j in enumerate(category_cols):
        if (drop == 'if_binary') & (len(col_value[i]) == 2):
            cate_cols_new.append(j)
        else:
            for f in col_value[i]:
                feature_name = str(j) + '_' + str(f)
                cate_cols_new.append(feature_name)
    return(cate_cols_new)

def result_df(model, X_train, y_train, X_test, y_test, metrics=
              [accuracy_score, recall_score, precision_score, f1_score, roc_auc_score]):
    res_train = []
    res_test = []
    col_name = []
    for fun in metrics:
        res_train.append(fun(model.predict(X_train), y_train))
        res_test.append(fun(model.predict(X_test), y_test)) 
        col_name.append(fun.__name__)
    idx_name = ['train_eval', 'test_eval']
    res = pd.DataFrame([res_train, res_test], columns=col_name, index=idx_name)
    return res

def colName(ColumnTransformer, numeric_cols, category_cols):
    col_name = []
    col_value = ColumnTransformer.named_transformers_['cat'].categories_
    
    for i, j in enumerate(category_cols):
        if len(col_value[i]) == 2:
            col_name.append(j)
        else:
            for f in col_value[i]:
                feature_name = j + '_' + f
                col_name.append(feature_name)
    col_name.extend(numeric_cols)
    return(col_name)
def Cross_Combination(colSet, df):
    newDf_l = []
    col_name_l = []
    
    for col in colSet:
        for col_sub in colSet:
            if col == col_sub:
                continue
            else:
                col_name = col+'&'+col_sub
                newDf_l.append(pd.Series(df[col].astype('str')+'&'+df[col_sub].astype('str'), name=col_name))
                col_name_l.append(col_name)
    
    newDF = pd.concat(newDf_l, axis=1)
    return newDF, col_name_l

def IV(new_features, DataFrame, target):
    """new_features:新特征
       DataFrame:数据集
       target：标签的名称
    """
    count_result = DataFrame[target].value_counts().values
    
    def IV_cal(features_name, target, df_temp):
        IV_l = []
        for i in features_name:
            IV_temp_l = []
            for values in df_temp[i].unique():
                data_temp = df_temp[df_temp[i] == values][target]
                PB, PG = data_temp.value_counts().values / count_result
                IV_temp = (PG-PB) * np.log(PG/PB)
                IV_temp_l.append(IV_temp)
            IV_l.append(np.array(IV_temp_l).sum())
        return(IV_l)
            
    if type(new_features) == np.ndarray:
        features_name = ['new_features']
        new_features = pd.Series(new_features, name=features_name[0])
    elif type(new_features) == pd.Series:
        features_name = [new_features.name]
    else:
        features_name = new_features.columns

    df_temp = pd.concat([new_features, DataFrame], axis=1)
    df_temp = df_temp.loc[:, ~df_temp.columns.duplicated()]
    IV_l = IV_cal(features_name=features_name, target=target, df_temp=df_temp)

    res = pd.DataFrame(IV_l, columns=['IV'], index=features_name)
    return(res)

def Binary_Cross_Combination(colNames, features, OneHot=True):
    """
    分类变量两两交叉组合衍生函数
    
    :param colNames: 参与交叉组合衍生的列名称
    :param features: 原始数据集的特征矩阵
    :param OneHot: 是否进行独热编码
    
    :return：交叉衍生后的新特征和新列名称
    """
    
    # 创建空列表存储器
    colNames_new_l = []
    features_new_l = []
    
    # 提取需要进行交叉组合的特征
    features = features[colNames]
    
    # 逐个创造新特征名称、新特征
    for col_index, col_name in enumerate(colNames):
        for col_sub_index in range(col_index+1, len(colNames)):
            
            newNames = col_name + '&' + colNames[col_sub_index]
            colNames_new_l.append(newNames)
            
            newDF = pd.Series(features[col_name].astype('str')  
                              + '&'
                              + features[colNames[col_sub_index]].astype('str'), 
                              name=col_name)
            features_new_l.append(newDF)
    
    # 拼接新特征矩阵
    features_new = pd.concat(features_new_l, axis=1)
    features_new.columns = colNames_new_l
    colNames_new = colNames_new_l
    
    # 对新特征矩阵进行独热编码
    if OneHot == True:
        enc = preprocessing.OneHotEncoder()
        enc.fit_transform(features_new)
        colNames_new = cate_colName(enc, colNames_new_l, drop=None)
        features_new = pd.DataFrame(enc.fit_transform(features_new).toarray(), columns=colNames_new)
        
    return features_new, colNames_new


def Binary_Group_Statistics(keyCol, 
                            features, 
                            col_num=None, 
                            col_cat=None, 
                            num_stat=['mean', 'var', 'max', 'min', 'skew', 'median'], 
                            cat_stat=['mean', 'var', 'max', 'min', 'median', 'count', 'nunique'], 
                            quant=True):
    """
    双变量分组统计特征衍生函数
    
    :param keyCol: 分组参考的关键变量
    :param features: 原始数据集
    :param col_num: 参与衍生的连续型变量
    :param col_cat: 参与衍生的离散型变量
    :param num_stat: 连续变量分组统计量
    :param cat_num: 离散变量分组统计量  
    :param quant: 是否计算分位数  

    :return：交叉衍生后的新特征和新特征的名称
    """
    
    # 当输入的特征有连续型特征时
    if col_num != None:
        aggs_num = {}
        colNames = col_num
        
        # 创建agg方法所需字典
        for col in col_num:
            aggs_num[col] = num_stat 
            
        # 创建衍生特征名称列表
        cols_num = [keyCol]
        for key in aggs_num.keys():
            cols_num.extend([key+'_'+keyCol+'_'+stat for stat in aggs_num[key]])
            
        # 创建衍生特征df
        features_num_new = features[col_num+[keyCol]].groupby(keyCol).agg(aggs_num).reset_index()
        features_num_new.columns = cols_num 
        
        # 当输入的特征有连续型也有离散型特征时
        if col_cat != None:        
            aggs_cat = {}
            colNames = col_num + col_cat

            # 创建agg方法所需字典
            for col in col_cat:
                aggs_cat[col] = cat_stat

            # 创建衍生特征名称列表
            cols_cat = [keyCol]
            for key in aggs_cat.keys():
                cols_cat.extend([key+'_'+keyCol+'_'+stat for stat in aggs_cat[key]])    

            # 创建衍生特征df
            features_cat_new = features[col_cat+[keyCol]].groupby(keyCol).agg(aggs_cat).reset_index()
            features_cat_new.columns = cols_cat
    
            # 合并连续变量衍生结果与离散变量衍生结果
            df_temp = pd.merge(features_num_new, features_cat_new, how='left',on=keyCol)
            features_new = pd.merge(features[keyCol], df_temp, how='left',on=keyCol)
            features_new.loc[:, ~features_new.columns.duplicated()]
            colNames_new = cols_num + cols_cat
            colNames_new.remove(keyCol)
            colNames_new.remove(keyCol)
         
        # 当只有连续变量时
        else:
            # merge连续变量衍生结果与原始数据，然后删除重复列
            features_new = pd.merge(features[keyCol], features_num_new, how='left',on=keyCol)
            features_new.loc[:, ~features_new.columns.duplicated()]
            colNames_new = cols_num
            colNames_new.remove(keyCol)
    
    # 当没有输入连续变量时
    else:
        # 但存在分类变量时，即只有分类变量时
        if col_cat != None:
            aggs_cat = {}
            colNames = col_cat

            for col in col_cat:
                aggs_cat[col] = cat_stat

            cols_cat = [keyCol]
            for key in aggs_cat.keys():
                cols_cat.extend([key+'_'+keyCol+'_'+stat for stat in aggs_cat[key]])    

            features_cat_new = features[col_cat+[keyCol]].groupby(keyCol).agg(aggs_cat).reset_index()
            features_cat_new.columns = cols_cat            
             
            features_new = pd.merge(features[keyCol], features_cat_new, how='left',on=keyCol)
            features_new.loc[:, ~features_new.columns.duplicated()]
            colNames_new = cols_cat
            colNames_new.remove(keyCol) 
    
    if quant:
        # 定义四分位计算函数
        def q1(x):
            """
            下四分位数
            """
            return x.quantile(0.25)

        def q2(x):
            """
            上四分位数
            """
            return x.quantile(0.75)

        aggs = {}
        for col in colNames:
            aggs[col] = ['q1', 'q2']

        cols = [keyCol]
        for key in aggs.keys():
            cols.extend([key+'_'+keyCol+'_'+stat for stat in aggs[key]])    

        aggs = {}
        for col in colNames:
            aggs[col] = [q1, q2]    

        features_temp = features[colNames+[keyCol]].groupby(keyCol).agg(aggs).reset_index()
        features_temp.columns = cols

        features_new = pd.merge(features_new, features_temp, how='left',on=keyCol)
        features_new.loc[:, ~features_new.columns.duplicated()]
        colNames_new = colNames_new + cols
        colNames_new.remove(keyCol)     
    
    features_new.drop([keyCol], axis=1, inplace=True)
        
    return features_new, colNames_new

def Binary_PolynomialFeatures(colNames, degree, features):
    """
    连续变量两变量多项式衍生函数
    
    :param colNames: 参与多项式衍生的列名称
    :param degree: 多项式最高阶
    :param features: 原始数据集
    
    :return：交叉衍生后的新特征和新列名称
    """
    
    
    # 创建空列表存储器
    colNames_new_l = []
    features_new_l = []
    
    # 提取需要进行多项式衍生的特征
    features = features[colNames]
    
    # 逐个进行多项式特征组合
    for col_index, col_name in enumerate(colNames):
        for col_sub_index in range(col_index+1, len(colNames)):
            col_temp = [col_name] + [colNames[col_sub_index]]
            array_new_temp = PolynomialFeatures(degree=degree, include_bias=False).fit_transform(features[col_temp])
            features_new_l.append(pd.DataFrame(array_new_temp[:, 2:]))
    
            # 逐个创建衍生多项式特征的名称
            for deg in range(2, degree+1):
                for i in range(deg+1):
                    col_name_temp = col_temp[0] + '**' + str(deg-i) + '*'+ col_temp[1] + '**' + str(i)
                    colNames_new_l.append(col_name_temp)
            
    
    # 拼接新特征矩阵
    features_new = pd.concat(features_new_l, axis=1)
    features_new.columns = colNames_new_l
    colNames_new = colNames_new_l
    
    return features_new, colNames_new

def Group_Statistics_Extension(colNames, keyCol, features):
    """
    双变量分组统计二阶特征衍生函数
    
    :param colNames: 参与衍生的特征名称
    :param keyCol: 分组参考的关键变量
    :param features: 原始数据集
    
    :return：交叉衍生后的新特征和新列名称
    """
    
    # 定义四分位计算函数
    def q1(x):
        """
        下四分位数
        """
        return x.quantile(0.25)

    def q2(x):
        """
        上四分位数
        """
        return x.quantile(0.75)   
    
    # 一阶特征衍生
    # 先定义用于生成列名称的aggs
    aggs = {}    
    for col in colNames:
        aggs[col] = ['mean', 'var', 'median', 'q1', 'q2']       
    cols = [keyCol]
    for key in aggs.keys():
        cols.extend([key+'_'+keyCol+'_'+stat for stat in aggs[key]])

    # 再定义用于进行分组汇总的aggs
    aggs = {}   
    for col in colNames:
        aggs[col] = ['mean', 'var', 'median', q1, q2] 
           
    features_new = features[colNames+[keyCol]].groupby(keyCol).agg(aggs).reset_index()
    features_new.columns = cols
             
    features_new = pd.merge(features[keyCol], features_new, how='left',on=keyCol)
    features_new.loc[:, ~features_new.columns.duplicated()]
    colNames_new = cols
    colNames_new.remove(keyCol)
    col1 = colNames_new.copy()
    print(col1)
    
    # 二阶特征衍生
    # 流量平滑特征
    for col_temp in colNames:
        col = col_temp+'_'+keyCol+'_'+'mean'
        features_new[col_temp+'_dive1_'+col] = features_new[keyCol] / (features_new[col] + 1e-5)
        colNames_new.append(col_temp+'_dive1_'+col)
        col = col_temp+'_'+keyCol+'_'+'median'
        features_new[col_temp+'_dive2_'+col] = features_new[keyCol] / (features_new[col] + 1e-5)
        colNames_new.append(col_temp+'_dive2_'+col)
        
    # 黄金组合特征
    for col_temp in colNames:
        col = col_temp+'_'+keyCol+'_'+'mean'
        features_new[col_temp+'_minus1_'+col] = features_new[keyCol] - features_new[col] 
        colNames_new.append(col_temp+'_minus1_'+col)
        features_new[col_temp+'_minus2_'+col] = features_new[keyCol] - features_new[col] 
        colNames_new.append(col_temp+'_minus2_'+col)
        
    # 组内归一化特征
    for col_temp in colNames:
        col_mean = col_temp+'_'+keyCol+'_'+'mean'
        col_var = col_temp+'_'+keyCol+'_'+'var'
        features_new[col_temp+'_norm_'+keyCol] = (features_new[keyCol] - features_new[col_mean]) / (np.sqrt(features_new[col_var]) + 1e-5)      
        colNames_new.append(col_temp+'_norm_'+keyCol)
    
    # Gap特征
    for col_temp in colNames:
        col_q1 = col_temp+'_'+keyCol+'_'+'q1'
        col_q2 = col_temp+'_'+keyCol+'_'+'q2'
        features_new[col_temp+'_gap_'+keyCol] = features_new[col_q2] - features_new[col_q1]  
        colNames_new.append(col_temp+'_gap_'+keyCol)
        
    # 数据倾斜特征
    for col_temp in colNames:
        col_mean = col_temp+'_'+keyCol+'_'+'mean'
        col_median = col_temp+'_'+keyCol+'_'+'median'
        features_new[col_temp+'_mag1_'+keyCol] = features_new[col_median] - features_new[col_mean]    
        colNames_new.append(col_temp+'_mag1_'+keyCol)
        features_new[col_temp+'_mag2_'+keyCol] = features_new[col_median] / (features_new[col_mean] + 1e-5)
        colNames_new.append(col_temp+'_mag2_'+keyCol)
        
    # 变异系数
    for col_temp in colNames:
        col_mean = col_temp+'_'+keyCol+'_'+'mean'
        col_var = col_temp+'_'+keyCol+'_'+'var'
        features_new[col_temp+'_cv_'+keyCol] = np.sqrt(features_new[col_var]) / (features_new[col_mean] + 1e-5)
        colNames_new.append(col_temp+'_cv_'+keyCol)

    features_new.drop([keyCol], axis=1, inplace=True)
    features_new.drop(col1, axis=1, inplace=True)
    colNames_new = list(features_new.columns)
    
    return features_new, colNames_new

def Multi_Cross_Combination(colNames, features, OneHot=True):
    """
    多变量组合交叉衍生
    
    :param colNames: 参与组合交叉衍生的列名称
    :param features: 原始数据集
    :param OneHot: 是否进行独热编码
    
    :return：交叉衍生后的新特征和新列名称
    """
    
    
    # 创建组合特征
    colNames_new = '&'.join([str(i) for i in colNames])
    features_new = features[colNames[0]].astype('str')

    for col in colNames[1:]:
        features_new = features_new + '&' + features[col].astype('str') 
    
    # 将组合特征转化为DataFrame
    features_new = pd.DataFrame(features_new, columns=[colNames_new])
    
    # 对新的特征列进行独热编码
    if OneHot == True:
        enc = preprocessing.OneHotEncoder()
        enc.fit_transform(features_new)
        colNames_new = cate_colName(enc, [colNames_new], drop=None)
        features_new = pd.DataFrame(enc.fit_transform(features_new).toarray(), columns=colNames_new)
        
    return features_new, colNames_new

def Multi_Group_Statistics(keyCol, 
                           features, 
                           col_num=None, 
                           col_cat=None, 
                           num_stat=['mean', 'var', 'max', 'min', 'skew', 'median'], 
                           cat_stat=['mean', 'var', 'max', 'min', 'median', 'count', 'nunique'], 
                           quant=True):
    """
    多变量分组统计特征衍生函数
    
    :param keyCol: 分组参考的关键变量
    :param features: 原始数据集
    :param col_num: 参与衍生的连续型变量
    :param col_cat: 参与衍生的离散型变量
    :param num_stat: 连续变量分组统计量
    :param cat_num: 离散变量分组统计量  
    :param quant: 是否计算分位数  

    :return：交叉衍生后的新特征和新特征的名称
    """
    # 生成原数据合并的主键
    features_key1, col1 = Multi_Cross_Combination(keyCol, features, OneHot=False)
    
    # 当输入的特征有连续型特征时
    if col_num != None:
        aggs_num = {}
        colNames = col_num
        
        # 创建agg方法所需字典
        for col in col_num:
            aggs_num[col] = num_stat 
            
        # 创建衍生特征名称列表
        cols_num = keyCol.copy()

        for key in aggs_num.keys():
            cols_num.extend([key+'_'+col1+'_'+stat for stat in aggs_num[key]])
            
        # 创建衍生特征df
        features_num_new = features[col_num+keyCol].groupby(keyCol).agg(aggs_num).reset_index()
        features_num_new.columns = cols_num 
        
        # 生成主键
        features_key2, col2 = Multi_Cross_Combination(keyCol, features_num_new, OneHot=False)
        
        # 创建包含合并主键的数据集
        features_num_new = pd.concat([features_key2, features_num_new], axis=1)
        
        
        # 当输入的特征有连续型也有离散型特征时
        if col_cat != None:        
            aggs_cat = {}
            colNames = col_num + col_cat

            # 创建agg方法所需字典
            for col in col_cat:
                aggs_cat[col] = cat_stat

            # 创建衍生特征名称列表
            cols_cat = keyCol.copy()
            
            for key in aggs_cat.keys():
                cols_cat.extend([key+'_'+col1+'_'+stat for stat in aggs_cat[key]])    

            # 创建衍生特征df
            features_cat_new = features[col_cat+keyCol].groupby(keyCol).agg(aggs_cat).reset_index()
            features_cat_new.columns = cols_cat
            
            # 生成主键
            features_key3, col3 = Multi_Cross_Combination(keyCol, features_cat_new, OneHot=False)

            # 创建包含合并主键的数据集
            features_cat_new = pd.concat([features_key3, features_cat_new], axis=1)            
    
    
            # 合并连续变量衍生结果与离散变量衍生结果
            # 合并新的特征矩阵
            df_temp = pd.concat([features_num_new, features_cat_new], axis=1)
            df_temp = df_temp.loc[:, ~df_temp.columns.duplicated()]
            # 将新的特征矩阵与原始数据集合并
            features_new = pd.merge(features_key1, df_temp, how='left',on=col1)
         
        
        # 当只有连续变量时
        else:
            # merge连续变量衍生结果与原始数据，然后删除重复列
            features_new = pd.merge(features_key1, features_num_new, how='left',on=col1)
            features_new = features_new.loc[:, ~features_new.columns.duplicated()]
    
    # 当没有输入连续变量时
    else:
        # 但存在分类变量时，即只有分类变量时
        if col_cat != None:
            aggs_cat = {}
            colNames = col_cat

            for col in col_cat:
                aggs_cat[col] = cat_stat

            cols_cat = keyCol.copy()
            
            for key in aggs_cat.keys():
                cols_cat.extend([key+'_'+col1+'_'+stat for stat in aggs_cat[key]])    

            features_cat_new = features[col_cat+keyCol].groupby(keyCol).agg(aggs_cat).reset_index()
            features_cat_new.columns = cols_cat            
             
            features_new = pd.merge(features_key1, features_cat_new, how='left',on=col1)
            features_new = features_new.loc[:, ~features_new.columns.duplicated()]
    
    if quant:
        # 定义四分位计算函数
        def q1(x):
            """
            下四分位数
            """
            return x.quantile(0.25)

        def q2(x):
            """
            上四分位数
            """
            return x.quantile(0.75)

        aggs = {}
        for col in colNames:
            aggs[col] = ['q1', 'q2']

        cols = keyCol.copy()
        
        for key in aggs.keys():
            cols.extend([key+'_'+col1+'_'+stat for stat in aggs[key]])    

        aggs = {}
        for col in colNames:
            aggs[col] = [q1, q2]    

        features_temp = features[colNames+keyCol].groupby(keyCol).agg(aggs).reset_index()
        features_temp.columns = cols
        features_new.drop(keyCol, axis=1, inplace=True)
    
        # 生成主键
        features_key4, col4 = Multi_Cross_Combination(keyCol, features_temp, OneHot=False)
        
        # 创建包含合并主键的数据集
        features_temp = pd.concat([features_key4, features_temp], axis=1)        

        # 合并新特征矩阵
        features_new = pd.merge(features_new, features_temp, how='left',on=col1)
        features_new = features_new.loc[:, ~features_new.columns.duplicated()]
  

    features_new.drop(keyCol+[col1], axis=1, inplace=True)
    colNames_new = list(features_new.columns)
    
    return features_new, colNames_new

def Multi_PolynomialFeatures(colNames, degree, features):
    """
    连续变量多变量多项式衍生函数
    
    :param colNames: 参与衍生的列名称
    :param degree: 多项式最高阶
    :param features: 原始数据集
    
    :return：交叉衍生后的新特征和新列名称
    """
    
    
    # 创建空列表容器
    colNames_new_l = []
    
    # 计算带入多项式计算的特征数
    n = len(colNames)
    
    # 提取需要进行多项式衍生的特征
    features = features[colNames]
    
    # 进行多项式特征组合
    array_new_temp = PolynomialFeatures(degree=degree, include_bias=False).fit_transform(features)
    # 选取衍生的特征
    array_new_temp = array_new_temp[:, n:]
    
    
    # 创建列名称列表
    deg = 2
    while deg <= degree:
        m = 1
        a1 = range(deg, -1, -1)
        a2 = []
        while m < n:
            a1 = list(product(a1, range(deg, -1, -1)))
            if m > 1:
                for i in a1:
                    i_temp = list(i[0])
                    i_temp.append(i[1])
                    a2.append(i_temp)
                a1 = a2.copy()
                a2 = []    
            m += 1
            
        a1 = np.array(a1)
        a3 = a1[a1.sum(1) == deg]
        
        for i in a3:
            colNames_new_l.append('&'.join(colNames) + '_' + ''.join([str(i) for i in i]))    
        
        deg += 1
    
    # 拼接新特征矩阵
    features_new = pd.DataFrame(array_new_temp, columns=colNames_new_l)
    colNames_new = colNames_new_l
    
    return features_new, colNames_new

def Features_Padding(features_train_new, 
                     features_test_new, 
                     colNames_train_new, 
                     colNames_test_new):
    """
    特征零值填补函数
    
    :param features_train_new: 训练集衍生特征
    :param features_test_new: 测试集衍生特征
    :param colNames_train_new: 训练集衍生列名称
    :param colNames_test_new: 测试集衍生列名称
    
    :return：0值填补后的新特征和特征名称
    """
    if len(colNames_train_new) > len(colNames_test_new):
        sub_colNames = list(set(colNames_train_new) - set(colNames_test_new))
        
        for col in sub_colNames:
            features_test_new[col] = 0
        
        features_test_new = features_test_new[colNames_train_new]
        colNames_test_new = list(features_test_new.columns)
            
    elif len(colNames_train_new) < len(colNames_test_new):
        sub_colNames = list(set(colNames_test_new) - set(colNames_train_new))
        
        for col in sub_colNames:
            features_train_new[col] = 0
        
        features_train_new = features_train_new[colNames_test_new]
        colNames_train_new = list(features_train_new.columns)    
    assert colNames_train_new  == colNames_test_new
    return features_train_new, features_test_new, colNames_train_new, colNames_test_new        



def test_features(keyCol,
                  X_train, 
                  X_test,
                  features_train_new,
                  multi=False):
    """
    测试集特征填补函数
    
    :param keyCol: 分组参考的关键变量
    :param X_train: 训练集特征
    :param X_test: 测试集特征
    :param features_train_new: 训练集衍生特征
    :param multi: 是否多变量参与分组
    
    :return：分组统计衍生后的新特征和新特征的名称
    """
    
    # 创建主键
    # 创建带有主键的训练集衍生特征df
    # 创建只包含主键的test_key
    if multi == False:
        keyCol = keyCol
        features_train_new[keyCol] = X_train[keyCol].reset_index()[keyCol]
        test_key = pd.DataFrame(X_test[keyCol])
    else:
        train_key, train_col = Multi_Cross_Combination(colNames=keyCol, features=X_train, OneHot=False)
        test_key, test_col = Multi_Cross_Combination(colNames=keyCol, features=X_test, OneHot=False)
        assert train_col == test_col
        keyCol = train_col
        features_train_new[keyCol] = train_key[train_col].reset_index()[train_col]
        
    # 利用groupby进行去重
    features_test_or = features_train_new.groupby(keyCol).mean().reset_index()
    
    # 和测试集进行拼接
    features_test_new = pd.merge(test_key, features_test_or, on=keyCol, how='left')
    
    # 删除keyCol列，只保留新衍生的列
    features_test_new.drop([keyCol], axis=1, inplace=True)
    features_train_new.drop([keyCol], axis=1, inplace=True)
    
    # 输出列名称
    colNames_train_new = list(features_train_new.columns)
    colNames_test_new = list(features_test_new.columns)
    
    return features_train_new, features_test_new, colNames_train_new, colNames_test_new



def Cross_Combination(colNames, 
                      X_train,
                      X_test,
                      multi=False,
                      OneHot=True):
    """
    交叉组合特征衍生函数
    
    :param colNames: 参与交叉衍生的列名称
    :param X_train: 训练集特征
    :param X_test: 测试集特征
    :param multi: 是否进行多变量交叉组合
    :param OneHot: 是否进行独热编码
    
    :return：交叉衍生后的新特征和特征名称
    """
    # 首先，训练集和测试集单独进行交叉组合特征衍生
    if multi == False:
        features_train_new, colNames_train_new = Binary_Cross_Combination(colNames=colNames, features=X_train, OneHot=OneHot)
        features_test_new, colNames_test_new = Binary_Cross_Combination(colNames=colNames, features=X_test, OneHot=OneHot)
    else:
        features_train_new, colNames_train_new = Multi_Cross_Combination(colNames=colNames, features=X_train, OneHot=OneHot)
        features_test_new, colNames_test_new = Multi_Cross_Combination(colNames=colNames, features=X_test, OneHot=OneHot)
        
    # 然后判断训练集和测试集的衍生特征是否存在差异
    if colNames_train_new != colNames_test_new:
        features_train_new, features_test_new, colNames_train_new, colNames_test_new = Features_Padding(features_train_new = features_train_new, 
                                                                                                        features_test_new = features_test_new, 
                                                                                                        colNames_train_new = colNames_train_new, 
                                                                                                        colNames_test_new = colNames_test_new)
    return features_train_new, features_test_new, colNames_train_new, colNames_test_new


def Polynomial_Features(colNames, 
                        degree, 
                        X_train, 
                        X_test, 
                        multi=False):   
    
    """
    多项式特征衍生函数
    
    :param colNames: 参与交叉衍生的列名称
    :param degree: 多项式最高阶
    :param X_train: 训练集特征
    :param X_test: 测试集特征
    :param multi: 是否进行多变量多项式组衍生
    
    :return：多项式衍生后的新特征和新列名称
    """
    if multi == False:
        features_train_new, colNames_train_new = Binary_PolynomialFeatures(colNames=colNames, degree=degree, features=X_train)
        features_test_new, colNames_test_new = Binary_PolynomialFeatures(colNames=colNames, degree=degree, features=X_test)
    else:
        features_train_new, colNames_train_new = Multi_PolynomialFeatures(colNames=colNames, degree=degree, features=X_train)
        features_test_new, colNames_test_new = Multi_PolynomialFeatures(colNames=colNames, degree=degree, features=X_test)
        
    assert colNames_train_new  == colNames_test_new
    return features_train_new, features_test_new, colNames_train_new, colNames_test_new


def Group_Statistics(keyCol, 
                     X_train, 
                     X_test, 
                     col_num=None, 
                     col_cat=None, 
                     num_stat=['mean', 'var', 'max', 'min', 'skew', 'median'], 
                     cat_stat=['mean', 'var', 'max', 'min', 'median', 'count', 'nunique'], 
                     quant=True, 
                     multi=False, 
                     extension=False):
    """
    分组统计特征衍生函数
    
    :param keyCol: 分组参考的关键变量
    :param X_train: 训练集特征
    :param X_test: 测试集特征
    :param col_num: 参与衍生的连续型变量
    :param col_cat: 参与衍生的离散型变量
    :param num_stat: 连续变量分组统计量
    :param cat_num: 离散变量分组统计量  
    :param quant: 是否计算分位数  
    :param multi: 是否进行多变量的分组统计特征衍生
    :param extension: 是否进行二阶特征衍生

    :return：分组统计衍生后的新特征和新特征的名称
    """
    
    # 进行训练集的特征衍生
    if multi == False:
        # 进行双变量的交叉衍生
        features_train_new, colNames_train_new = Binary_Group_Statistics(keyCol = keyCol, 
                                                                         features = X_train, 
                                                                         col_num = col_num, 
                                                                         col_cat = col_cat, 
                                                                         num_stat = num_stat, 
                                                                         cat_stat = cat_stat, 
                                                                         quant = quant)
        # 是否进一步进行二阶特征衍生
        if extension == True:
            if col_num == None:
                colNames = col_cat
            elif col_cat == None:
                colNames = col_num
            else:
                colNames = col_num + col_cat
                
            features_train_new_ex, colNames_train_new_ex = Group_Statistics_Extension(colNames = colNames,
                                                                                      keyCol = keyCol,
                                                                                      features = X_train)
            
            features_train_new = pd.concat([features_train_new, features_train_new_ex], axis=1)
            colNames_train_new.extend(colNames_train_new_ex)
            
        
    else:
        # 进行多变量的交叉衍生
        features_train_new, colNames_train_new = Multi_Group_Statistics(keyCol = keyCol, 
                                                                        features = X_train, 
                                                                        col_num = col_num, 
                                                                        col_cat = col_cat, 
                                                                        num_stat = num_stat, 
                                                                        cat_stat = cat_stat, 
                                                                        quant = quant)

    
    # 对测试集结果进行填补
    features_train_new, features_test_new, colNames_train_new, colNames_test_new = test_features(keyCol, 
                                                                                                 X_train, 
                                                                                                 X_test, 
                                                                                                 features_train_new,
                                                                                                 multi=multi)
    # 如果特征不一致，则进行0值填补
    # 对于分组统计特征来说一般不会出现该情况
    if colNames_train_new != colNames_test_new:
        features_train_new, features_test_new, colNames_train_new, colNames_test_new = Features_Padding(features_train_new = features_train_new, 
                                                                                                        features_test_new = features_test_new, 
                                                                                                        colNames_train_new = colNames_train_new, 
                                                                                                        colNames_test_new = colNames_test_new)
    
    
    assert colNames_train_new  == colNames_test_new
    return features_train_new, features_test_new, colNames_train_new, colNames_test_new


def Target_Encode(keyCol, 
                  X_train, 
                  y_train,
                  X_test, 
                  col_num=None, 
                  col_cat=None, 
                  num_stat=['mean', 'var', 'max', 'min', 'skew', 'median'], 
                  cat_stat=['mean', 'var', 'max', 'min', 'median', 'count', 'nunique'], 
                  quant=True, 
                  multi=False, 
                  extension=False,
                  n_splits=5, 
                  random_state=42):
    """
    目标编码
    
    :param keyCol: 分组参考的关键变量
    :param X_train: 训练集特征
    :param y_train: 训练集标签
    :param X_test: 测试集特征
    :param col_num: 参与衍生的连续型变量
    :param col_cat: 参与衍生的离散型变量
    :param num_stat: 连续变量分组统计量
    :param cat_num: 离散变量分组统计量  
    :param quant: 是否计算分位数  
    :param multi: 是否进行多变量的分组统计特征衍生
    :param extension: 是否进行二阶特征衍生
    :param n_splits: 进行几折交叉统计  
    :param random_state: 随机数种子  

    :return：目标编码后的新特征和新特征的名称
    """
        
    # 获取标签名称
    target = y_train.name
    # 合并同时带有特征和标签的完整训练集
    train = pd.concat([X_train, y_train], axis=1)
    
    folds = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # 每一折验证集的结果存储容器
    df_l = []
    
    # 进行交叉统计
    for trn_idx, val_idx in folds.split(train):
        trn_temp = train.iloc[trn_idx]
        val_temp = train.iloc[val_idx]
        trn_temp_new, val_temp_new, colNames_trn_temp_new, colNames_val_temp_new = Group_Statistics(keyCol, 
                                                                                                    X_train = trn_temp, 
                                                                                                    X_test = val_temp, 
                                                                                                    col_num = col_num, 
                                                                                                    col_cat = col_cat, 
                                                                                                    num_stat = num_stat, 
                                                                                                    cat_stat = cat_stat, 
                                                                                                    quant = quant, 
                                                                                                    multi = multi, 
                                                                                                    extension = extension)
        val_temp_new.index = val_temp.index
        df_l.append(val_temp_new)
    
    # 创建训练集的衍生特征
    features_train_new = pd.concat(df_l).sort_index(ascending=True) 
    colNames_train_new = [col + '_kfold' for col in features_train_new.columns]
    features_train_new.columns = colNames_train_new
    
    # 对测试集结果进行填补
    features_train_new, features_test_new, colNames_train_new, colNames_test_new = test_features(keyCol = keyCol, 
                                                                                                 X_train = X_train, 
                                                                                                 X_test = X_test, 
                                                                                                 features_train_new = features_train_new,
                                                                                                 multi = multi)
   
    # 如果特征不一致，则进行0值填补
    if colNames_train_new != colNames_test_new:
        features_train_new, features_test_new, colNames_train_new, colNames_test_new = Features_Padding(features_train_new = features_train_new, 
                                                                                                        features_test_new = features_test_new, 
                                                                                                        colNames_train_new = colNames_train_new, 
                                                                                                        colNames_test_new = colNames_test_new)
        
    assert colNames_train_new  == colNames_test_new
    return features_train_new, features_test_new, colNames_train_new, colNames_test_new



In [37]:
data.fillna(data.mean(), inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_23384\1474580016.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data.fillna(data.mean(), inplace=True)


# 数据准备

In [38]:
# 检查DataFrame中是否有任何NaN值
has_nan = data.isnull().values.any()

print(f"DataFrame contains NaN: {has_nan}")

DataFrame contains NaN: False


In [45]:
# 假设 data 是已经加载的DataFrame
# data = pd.read_csv('your_dataset.csv')  # 示例代码，假设您已经加载了数据

# 离散字段
category_cols = [
    "gender", "start_time_DI", "last_event_DI","final_cc_cname_DI","LoE_DI"
]

# 连续字段
numeric_cols = [
    "registered", "viewed",  "explored", "certified", "YoB",
    "nevents", "ndays_act", "nplay_video","nchapters","nforum_posts"
]
# 标签
Target = "grade"
# ID列
ID_col = ["course_id", "userid_DI"]

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11722 entries, 0 to 11721
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   course_id          11722 non-null  object 
 1   userid_DI          11722 non-null  object 
 2   registered         11722 non-null  int64  
 3   viewed             11722 non-null  int64  
 4   explored           11722 non-null  int64  
 5   certified          11722 non-null  int64  
 6   final_cc_cname_DI  11722 non-null  object 
 7   LoE_DI             11722 non-null  object 
 8   YoB                11722 non-null  int64  
 9   gender             11722 non-null  object 
 10  grade              11722 non-null  int64  
 11  start_time_DI      11722 non-null  object 
 12  last_event_DI      11722 non-null  object 
 13  nevents            11722 non-null  int64  
 14  ndays_act          11722 non-null  int64  
 15  nplay_video        11722 non-null  float64
 16  nchapters          117

In [47]:

# # 连续字段转化
# data[numeric_cols]= data[numeric_cols].astype(float)

# #缺失值填补
# si = SimpleImputer(missing_values = np.nan, strategy = "most_frequent")
# si.fit(data.loc[:,:].values.reshape(-1, 1))
# data.loc[:,:] = si.transform(data.loc[:,:].values.reshape(-1, 1))

# # 标签值手动转化 
# data['Target'].replace(to_replace=["Dropout","Enrolled"], value=1, inplace=True)
# data['Target'].replace(to_replace=["Graduate"],  value=0, inplace=True)

In [49]:
train, test = train_test_split(data, random_state=55914)
X_train = train.drop(columns=["grade"]).copy()
X_test = test.drop(columns=["grade"]).copy()

y_train = train['grade'].copy()
y_test = test['grade'].copy()

ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
ord_enc.fit(X_train[category_cols])
# ord_enc.fit(X_train[category_cols])

X_train_OE = pd.DataFrame(ord_enc.transform(X_train[category_cols]), columns=category_cols)
X_train_OE.index = X_train.index

X_train_OE = pd.concat([X_train_OE, X_train[numeric_cols]], axis=1)

X_test_OE = pd.DataFrame(ord_enc.transform(X_test[category_cols]), columns=category_cols)
X_test_OE.index = X_test.index
X_test_OE = pd.concat([X_test_OE, X_test[numeric_cols]], axis=1)



In [50]:
X_test_OE.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2931 entries, 10723 to 5117
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             2931 non-null   float64
 1   start_time_DI      2931 non-null   float64
 2   last_event_DI      2931 non-null   float64
 3   final_cc_cname_DI  2931 non-null   float64
 4   LoE_DI             2931 non-null   float64
 5   registered         2931 non-null   int64  
 6   viewed             2931 non-null   int64  
 7   explored           2931 non-null   int64  
 8   certified          2931 non-null   int64  
 9   YoB                2931 non-null   int64  
 10  nevents            2931 non-null   int64  
 11  ndays_act          2931 non-null   int64  
 12  nplay_video        2931 non-null   float64
 13  nchapters          2931 non-null   int64  
 14  nforum_posts       2931 non-null   int64  
dtypes: float64(6), int64(9)
memory usage: 366.4 KB


# 特征工程

## 多项式特征衍生

In [51]:
Poly_train, Poly_test, colNames_train_new, colNames_test_new = Polynomial_Features(numeric_cols, 
                                                                                   3, 
                                                                                   X_train_OE, 
                                                                                   X_test_OE, 
                                                                                   )

In [52]:
Poly_train.head()

registered**2*viewed**0  registered**1*viewed**1  registered**0*viewed**2  \
0                      1.0                      1.0                      1.0   
1                      1.0                      1.0                      1.0   
2                      1.0                      1.0                      1.0   
3                      1.0                      1.0                      1.0   
4                      1.0                      1.0                      1.0   

   registered**3*viewed**0  registered**2*viewed**1  registered**1*viewed**2  \
0                      1.0                      1.0                      1.0   
1                      1.0                      1.0                      1.0   
2                      1.0                      1.0                      1.0   
3                      1.0                      1.0                      1.0   
4                      1.0                      1.0                      1.0   

   registered**0*viewed**3  registered**2*explored**0  \
0                      1.0                        1.0   
1                      1.0                        1.0   
2                      1.0                        1.0   
3                      1.0                        1.0   
4                      1.0                        1.0   

   registered**1*explored**1  registered**0*explored**2  ...  \
0                        0.0                        0.0  ...   
1                        1.0                        1.0  ...   
2                        0.0                        0.0  ...   
3                        0.0                        0.0  ...   
4                        0.0                        0.0  ...   

   nplay_video**2*nforum_posts**1  nplay_video**1*nforum_posts**2  \
0                             0.0                             0.0   
1                             0.0                             0.0   
2                             0.0                             0.0   
3                             0.0                             0.0   
4                             0.0                             0.0   

   nplay_video**0*nforum_posts**3  nchapters**2*nforum_posts**0  \
0                             0.0                          16.0   
1                             0.0                          81.0   
2                             0.0                         225.0   
3                             0.0                           9.0   
4                             0.0                           4.0   

   nchapters**1*nforum_posts**1  nchapters**0*nforum_posts**2  \
0                           0.0                           0.0   
1                           0.0                           0.0   
2                           0.0                           0.0   
3                           0.0                           0.0   
4                           0.0                           0.0   

   nchapters**3*nforum_posts**0  nchapters**2*nforum_posts**1  \
0                          64.0                           0.0   
1                         729.0                           0.0   
2                        3375.0                           0.0   
3                          27.0                           0.0   
4                           8.0                           0.0   

   nchapters**1*nforum_posts**2  nchapters**0*nforum_posts**3  
0                           0.0                           0.0  
1                           0.0                           0.0  
2                           0.0                           0.0  
3                           0.0                           0.0  
4                           0.0                           0.0  

[5 rows x 315 columns]

In [53]:
Poly_train.shape

(8791, 315)

## 多项式特征衍生的特征筛选

In [54]:
#方差过滤

In [55]:
sel = VarianceThreshold()
sel.fit(Poly_train)

VarianceThreshold()

In [56]:
Poly_cols = Poly_train.columns[sel.variances_ > 0]
Poly_cols = list(Poly_cols)
len(Poly_cols)

276

In [57]:
Poly_train[Poly_cols].to_csv('data/X_train_Poly.csv', index=False)
Poly_test[Poly_cols].to_csv('data/X_test_Poly.csv', index=False)

## 分组统计特征衍生

In [58]:
#KeyCol筛选

In [59]:
cat_all = category_cols.copy()

In [60]:
MI = mutual_info_classif(X_train_OE[cat_all], y_train, discrete_features=True, random_state=22) #互信息法
MI

array([0.00424126, 0.03440512, 0.20012993, 0.01070974, 0.00312486])

In [61]:
MI_select_cols = []
MI_threshold = MI.mean() * 1

for MIvalue, colname in zip(MI, cat_all):
    if MIvalue > MI_threshold:
        MI_select_cols.append(colname)

print(len(MI_select_cols))        
MI_select_cols

1


['last_event_DI']

In [62]:
keycol = MI_select_cols
keycol

['last_event_DI']

In [63]:
# 创建一个未被选中离散变量的list
cat_rest = []
for col in cat_all:
    if col not in keycol:
        cat_rest.append(col)

cat_rest

['gender', 'start_time_DI', 'final_cc_cname_DI', 'LoE_DI']

In [64]:
#分组统计特征衍生

In [65]:
# 创建容器
col_temp = keycol.copy()
GroupStat_train = pd.DataFrame()
GroupStat_test = pd.DataFrame()

for i in range(len(col_temp)):
    keyCol = col_temp.pop(i)
    features_train1, features_test1, colNames_train, colNames_test = Group_Statistics(keyCol,
                                                                                      X_train_OE,
                                                                                      X_test_OE,
                                                                                      col_num=numeric_cols,
                                                                                      col_cat=col_temp+cat_rest, 
                                                                                      extension=True)
    
    GroupStat_train = pd.concat([GroupStat_train, features_train1],axis=1)
    GroupStat_test = pd.concat([GroupStat_test, features_test1],axis=1)
    
    col_temp = keycol.copy()

['registered_last_event_DI_mean', 'registered_last_event_DI_var', 'registered_last_event_DI_median', 'registered_last_event_DI_q1', 'registered_last_event_DI_q2', 'viewed_last_event_DI_mean', 'viewed_last_event_DI_var', 'viewed_last_event_DI_median', 'viewed_last_event_DI_q1', 'viewed_last_event_DI_q2', 'explored_last_event_DI_mean', 'explored_last_event_DI_var', 'explored_last_event_DI_median', 'explored_last_event_DI_q1', 'explored_last_event_DI_q2', 'certified_last_event_DI_mean', 'certified_last_event_DI_var', 'certified_last_event_DI_median', 'certified_last_event_DI_q1', 'certified_last_event_DI_q2', 'YoB_last_event_DI_mean', 'YoB_last_event_DI_var', 'YoB_last_event_DI_median', 'YoB_last_event_DI_q1', 'YoB_last_event_DI_q2', 'nevents_last_event_DI_mean', 'nevents_last_event_DI_var', 'nevents_last_event_DI_median', 'nevents_last_event_DI_q1', 'nevents_last_event_DI_q2', 'ndays_act_last_event_DI_mean', 'ndays_act_last_event_DI_var', 'ndays_act_last_event_DI_median', 'ndays_act_last

C:\Users\DELL\AppData\Local\Temp\ipykernel_23384\3852591749.py:403: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_new[col_temp+'_mag2_'+keyCol] = features_new[col_median] / (features_new[col_mean] + 1e-5)
C:\Users\DELL\AppData\Local\Temp\ipykernel_23384\3852591749.py:401: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_new[col_temp+'_mag1_'+keyCol] = features_new[col_median] - features_new[col_mean]
C:\Users\DELL\AppData\Local\Temp\ipykernel_23384\3852591749.py:403: PerformanceWarning: DataFrame is highly fragmen

In [66]:
GroupStat_train.head()

registered_last_event_DI_mean  registered_last_event_DI_var  \
0                            1.0                           0.0   
1                            1.0                           0.0   
2                            1.0                           0.0   
3                            1.0                           0.0   
4                            1.0                           0.0   

   registered_last_event_DI_max  registered_last_event_DI_min  \
0                             1                             1   
1                             1                             1   
2                             1                             1   
3                             1                             1   
4                             1                             1   

   registered_last_event_DI_skew  registered_last_event_DI_median  \
0                            0.0                              1.0   
1                            0.0                              1.0   
2                            0.0                              1.0   
3                            0.0                              1.0   
4                            0.0                              1.0   

   viewed_last_event_DI_mean  viewed_last_event_DI_var  \
0                        1.0                       0.0   
1                        1.0                       0.0   
2                        1.0                       0.0   
3                        1.0                       0.0   
4                        1.0                       0.0   

   viewed_last_event_DI_max  viewed_last_event_DI_min  ...  \
0                         1                         1  ...   
1                         1                         1  ...   
2                         1                         1  ...   
3                         1                         1  ...   
4                         1                         1  ...   

   YoB_cv_last_event_DI  nevents_cv_last_event_DI  ndays_act_cv_last_event_DI  \
0              0.007062                  2.391349                    1.036231   
1              0.005063                  0.529356                    0.668282   
2              0.005063                  0.529356                    0.668282   
3              0.004594                  2.105250                    1.317072   
4              0.007877                  0.985245                    0.987577   

   nplay_video_cv_last_event_DI  nchapters_cv_last_event_DI  \
0                      1.799396                    1.000150   
1                      0.368630                    0.447049   
2                      0.368630                    0.447049   
3                      1.074656                    0.812190   
4                      0.632475                    0.922696   

   nforum_posts_cv_last_event_DI  gender_cv_last_event_DI  \
0                       0.000000                 1.019784   
1                       3.155856                 0.799558   
2                       3.155856                 0.799558   
3                       3.671200                 0.819147   
4                       0.000000                 0.985165   

   start_time_DI_cv_last_event_DI  final_cc_cname_DI_cv_last_event_DI  \
0                        0.686194                            0.638960   
1                        0.545974                            0.517480   
2                        0.545974                            0.517480   
3                        0.517981                            0.375402   
4                        0.425756                            0.401167   

   LoE_DI_cv_last_event_DI  
0                 1.478835  
1                 0.823164  
2                 0.823164  
3                 1.015371  
4                 0.854474  

[5 rows x 242 columns]

In [67]:
GroupStat_train.shape

(8791, 242)

## 分组统计特征衍生的特征筛选

In [68]:
#方差过滤

In [69]:
sel = VarianceThreshold()
sel.fit(GroupStat_train)

VarianceThreshold()

In [70]:
GroupStat_cols = list(GroupStat_train.columns[sel.variances_ > 0])
len(GroupStat_cols)

214

In [71]:
y_train.shape

(8791,)

In [72]:
GroupStat_train.shape

(8791, 242)

In [73]:
imputer = SimpleImputer(strategy='mean')  # 或者 'median', 'most_frequent'
GroupStat_train[GroupStat_cols] = imputer.fit_transform(GroupStat_train[GroupStat_cols])

In [74]:
#方差分析

In [75]:
f_classif_p = f_classif(GroupStat_train[GroupStat_cols], y_train)[1]

In [76]:
f_classif_GroupStat_cols = []

for pValue, colname in zip(f_classif_p, GroupStat_cols):
    if pValue < 0.000000000001:
        f_classif_GroupStat_cols.append(colname)

print(len(f_classif_GroupStat_cols))

161


In [77]:
#互信息法

In [78]:
MI = mutual_info_classif(GroupStat_train[GroupStat_cols], y_train, random_state=22)
MI

array([0.16924779, 0.14954032, 0.1032261 , 0.00802563, 0.13883287,
       0.10876026, 0.19601986, 0.16789202, 0.11435871, 0.        ,
       0.15687529, 0.13318968, 0.17181295, 0.17984958, 0.04695245,
       0.03490824, 0.18386952, 0.01817423, 0.17990539, 0.17991708,
       0.18410462, 0.11436866, 0.18638461, 0.17599903, 0.18569662,
       0.1936537 , 0.15125518, 0.01488335, 0.18393393, 0.13235787,
       0.18436043, 0.18787191, 0.14365374, 0.02840402, 0.18286314,
       0.13051323, 0.17504116, 0.18373636, 0.14703448, 0.03014107,
       0.18507871, 0.16005364, 0.10630307, 0.13889712, 0.02149242,
       0.12625574, 0.13351746, 0.12740244, 0.        , 0.00300028,
       0.03312214, 0.10760782, 0.00358308, 0.18407345, 0.18868878,
       0.04756325, 0.05657493, 0.1083246 , 0.09891957, 0.08858327,
       0.17530793, 0.18536058, 0.02353229, 0.01602133, 0.02865529,
       0.10930353, 0.06144396, 0.1392638 , 0.17607499, 0.00677629,
       0.        , 0.02743978, 0.11067749, 0.02559023, 0.08362

In [79]:
MI.mean()

0.14120729604092638

In [80]:
MI_threshold = MI.mean() * 0.2
MI_threshold

0.028241459208185278

In [81]:
MI_GroupStat_cols = []

for MIvalue, colname in zip(MI, GroupStat_cols):
    if MIvalue > MI_threshold:
        MI_GroupStat_cols.append(colname)

print(len(MI_GroupStat_cols))   

190


In [82]:
GroupStat_cols_select = list(set(f_classif_GroupStat_cols) & set(MI_GroupStat_cols))
len(GroupStat_cols_select)

149

In [83]:
GroupStat_train[GroupStat_cols_select]

start_time_DI_dive1_start_time_DI_last_event_DI_mean  \
0                                              1.496845      
1                                              1.742066      
2                                              1.742066      
3                                              1.039031      
4                                              1.099001      
...                                                 ...      
8786                                           1.663552      
8787                                           0.509046      
8788                                           1.750411      
8789                                           0.489385      
8790                                           0.079767      

      final_cc_cname_DI_mag2_last_event_DI  nplay_video_last_event_DI_max  \
0                                 0.932735                    3807.000000   
1                                 0.914185                     437.896054   
2                                 0.914185                     437.896054   
3                                 1.017377                    1516.000000   
4                                 1.028455                     477.000000   
...                                    ...                            ...   
8786                              1.015037                     583.000000   
8787                              0.917910                    1178.000000   
8788                              0.962771                    4312.000000   
8789                              0.999999                     791.000000   
8790                              0.916841                   17314.000000   

      ndays_act_last_event_DI_max  gender_last_event_DI_var  \
0                            64.0                  0.260000   
1                           112.0                  0.239505   
2                           112.0                  0.239505   
3                            68.0                  0.247354   
4                            52.0                  0.257576   
...                           ...                       ...   
8786                         63.0                  0.229550   
8787                         28.0                  0.265152   
8788                        116.0                  0.242347   
8789                         16.0                  0.257353   
8790                         91.0                  0.245394   

      nevents_cv_last_event_DI  explored_mag2_last_event_DI  \
0                     2.391349                     0.000000   
1                     0.529356                     1.148502   
2                     0.529356                     1.148502   
3                     2.105250                     0.000000   
4                     0.985245                     0.000000   
...                        ...                          ...   
8786                  0.767339                     1.190462   
8787                  0.799921                     0.999980   
8788                  1.100255                     1.324307   
8789                  1.199044                     0.000000   
8790                  0.783901                     1.044575   

      YoB_minus2_YoB_last_event_DI_mean  start_time_DI_last_event_DI_nunique  \
0                          -1762.884615                                 26.0   
1                          -1710.672414                                 93.0   
2                          -1710.672414                                 93.0   
3                          -1792.750000                                 22.0   
4                          -1761.757576                                 31.0   
...                                 ...                                  ...   
8786                       -1710.266667                                 59.0   
8787                       -1913.500000                                 10.0   
8788                       -1676.653061                                 42.0   
8789                  

In [84]:
GroupStat_train[GroupStat_cols_select].to_csv('data/X_train_GroupStat.csv', index=False)
GroupStat_test[GroupStat_cols_select].to_csv('data/X_test_GroupStat.csv', index=False)

In [85]:
# 定义标签
col_cat = [target]
print(col_cat)

# 创建容器
col_temp = cat_all.copy()
TarEnc_train = pd.DataFrame()
TarEnc_test = pd.DataFrame()

for keyCol in col_temp:
    features_train1, features_test1, colNames_train_new, colNames_test_new = Target_Encode(keyCol, 
                                                                                           X_train_OE, 
                                                                                           y_train,
                                                                                           X_test_OE, 
                                                                                           col_cat=col_cat, 
                                                                                           extension=True)
    
    TarEnc_train = pd.concat([TarEnc_train, features_train1],axis=1)
    TarEnc_test = pd.concat([TarEnc_test, features_test1],axis=1)
    
    col_temp = cat_all.copy()

['grade']
['grade_gender_mean', 'grade_gender_var', 'grade_gender_median', 'grade_gender_q1', 'grade_gender_q2']
['grade_gender_mean', 'grade_gender_var', 'grade_gender_median', 'grade_gender_q1', 'grade_gender_q2']
['grade_gender_mean', 'grade_gender_var', 'grade_gender_median', 'grade_gender_q1', 'grade_gender_q2']
['grade_gender_mean', 'grade_gender_var', 'grade_gender_median', 'grade_gender_q1', 'grade_gender_q2']
['grade_gender_mean', 'grade_gender_var', 'grade_gender_median', 'grade_gender_q1', 'grade_gender_q2']
['grade_start_time_DI_mean', 'grade_start_time_DI_var', 'grade_start_time_DI_median', 'grade_start_time_DI_q1', 'grade_start_time_DI_q2']
['grade_start_time_DI_mean', 'grade_start_time_DI_var', 'grade_start_time_DI_median', 'grade_start_time_DI_q1', 'grade_start_time_DI_q2']
['grade_start_time_DI_mean', 'grade_start_time_DI_var', 'grade_start_time_DI_median', 'grade_start_time_DI_q1', 'grade_start_time_DI_q2']
['grade_start_time_DI_mean', 'grade_start_time_DI_var', 'grad

In [86]:
TarEnc_train.head()

grade_gender_mean_kfold  grade_gender_var_kfold  grade_gender_max_kfold  \
0                 0.561367                0.246320                     1.0   
4                 0.472963                0.249329                     1.0   
5                 0.567897                0.245475                     1.0   
6                 0.567897                0.245475                     1.0   
7                 0.474475                0.249408                     1.0   

   grade_gender_min_kfold  grade_gender_median_kfold  \
0                     0.0                        1.0   
4                     0.0                        0.0   
5                     0.0                        1.0   
6                     0.0                        1.0   
7                     0.0                        0.0   

   grade_gender_count_kfold  grade_gender_nunique_kfold  \
0                    2868.0                         2.0   
4                    4161.0                         2.0   
5                    2872.0                         2.0   
6                    2872.0                         2.0   
7                    4192.0                         2.0   

   grade_gender_q1_kfold  grade_gender_q2_kfold  \
0                    0.0                    1.0   
4                    0.0                    1.0   
5                    0.0                    1.0   
6                    0.0                    1.0   
7                    0.0                    1.0   

   grade_dive1_grade_gender_mean_kfold  ...  grade_LoE_DI_q2_kfold  \
0                             0.000000  ...                    1.0   
4                             2.114285  ...                    1.0   
5                             0.000000  ...                    1.0   
6                             0.000000  ...                    1.0   
7                             2.107547  ...                    1.0   

   grade_dive1_grade_LoE_DI_mean_kfold  grade_dive2_grade_LoE_DI_median_kfold  \
0                             1.680823                                0.99999   
4                             5.791991                                2.99997   
5                             0.000000                                0.00000   
6                             0.000000                                0.00000   
7                             6.004926                           300000.00000   

   grade_minus1_grade_LoE_DI_mean_kfold  grade_minus2_grade_LoE_DI_mean_kfold  \
0                              0.405063                              0.405063   
4                              2.482053                              2.482053   
5                             -0.535008                             -0.535008   
6                             -0.535008                             -0.535008   
7                              2.500420                              2.500420   

   grade_norm_LoE_DI_kfold  grade_gap_LoE_DI_kfold  grade_mag1_LoE_DI_kfold  \
0                 0.824075                     1.0                 0.405063   
4                 4.966172                     1.0                 0.482053   
5                -1.072422                     1.0                 0.464992   
6                -1.072422                     1.0                 0.464992   
7                 4.999691                     1.0                -0.499580   

   grade_mag2_LoE_DI_kfold  grade_cv_LoE_DI_kfold  
0                 1.680823               0.826170  
4                 1.930664               0.964911  
5                 1.869097               0.932433  
6                 1.869097               0.932433  
7                 0.000000               1.001031  

[5 rows x 90 columns]

In [87]:
TarEnc_train.shape

(8791, 90)

In [88]:
sel = VarianceThreshold()
sel.fit(TarEnc_train)

VarianceThreshold()

In [89]:
TarEnc_cols = list(TarEnc_train.columns[sel.variances_ > 0])
len(TarEnc_cols)

75

In [90]:
TarEnc_train[TarEnc_cols].reset_index(drop=True).to_csv('data/X_train_TarEnc.csv', index=False)
TarEnc_test[TarEnc_cols].reset_index(drop=True).to_csv('data/X_test_TarEnc.csv', index=False)

## 将新特征与旧特征组合起来

In [91]:
X_train_GroupStat = pd.read_csv('data/X_train_GroupStat.csv')
X_train_Poly = pd.read_csv('data/X_train_Poly.csv')
X_train_TarEnc = pd.read_csv('data/X_train_TarEnc.csv')
X_test_GroupStat = pd.read_csv('data/X_test_GroupStat.csv')
X_test_Poly = pd.read_csv('data/X_test_Poly.csv')
X_test_TarEnc = pd.read_csv('data/X_test_TarEnc.csv')

In [92]:
features_train_new = pd.concat([X_train_GroupStat,
                                X_train_Poly,
                                X_train_TarEnc,], axis=1)

In [93]:
features_train_new

start_time_DI_dive1_start_time_DI_last_event_DI_mean  \
0                                              1.496845      
1                                              1.742066      
2                                              1.742066      
3                                              1.039031      
4                                              1.099001      
...                                                 ...      
8786                                           1.663552      
8787                                           0.509046      
8788                                           1.750411      
8789                                           0.489385      
8790                                           0.079767      

      final_cc_cname_DI_mag2_last_event_DI  nplay_video_last_event_DI_max  \
0                                 0.932735                    3807.000000   
1                                 0.914185                     437.896054   
2                                 0.914185                     437.896054   
3                                 1.017377                    1516.000000   
4                                 1.028455                     477.000000   
...                                    ...                            ...   
8786                              1.015037                     583.000000   
8787                              0.917910                    1178.000000   
8788                              0.962771                    4312.000000   
8789                              0.999999                     791.000000   
8790                              0.916841                   17314.000000   

      ndays_act_last_event_DI_max  gender_last_event_DI_var  \
0                            64.0                  0.260000   
1                           112.0                  0.239505   
2                           112.0                  0.239505   
3                            68.0                  0.247354   
4                            52.0                  0.257576   
...                           ...                       ...   
8786                         63.0                  0.229550   
8787                         28.0                  0.265152   
8788                        116.0                  0.242347   
8789                         16.0                  0.257353   
8790                         91.0                  0.245394   

      nevents_cv_last_event_DI  explored_mag2_last_event_DI  \
0                     2.391349                     0.000000   
1                     0.529356                     1.148502   
2                     0.529356                     1.148502   
3                     2.105250                     0.000000   
4                     0.985245                     0.000000   
...                        ...                          ...   
8786                  0.767339                     1.190462   
8787                  0.799921                     0.999980   
8788                  1.100255                     1.324307   
8789                  1.199044                     0.000000   
8790                  0.783901                     1.044575   

      YoB_minus2_YoB_last_event_DI_mean  start_time_DI_last_event_DI_nunique  \
0                          -1762.884615                                 26.0   
1                          -1710.672414                                 93.0   
2                          -1710.672414                                 93.0   
3                          -1792.750000                                 22.0   
4                          -1761.757576                                 31.0   
...                                 ...                                  ...   
8786                       -1710.266667                                 59.0   
8787                       -1913.500000                                 10.0   
8788                       -1676.653061                                 42.0   
8789                  

In [94]:
features_test_new = pd.concat([X_test_GroupStat,
                               X_test_Poly,
                               X_test_TarEnc,], axis=1)

In [95]:
features_test_new

start_time_DI_dive1_start_time_DI_last_event_DI_mean  \
0                                              0.079767      
1                                              1.288930      
2                                              1.287020      
3                                              1.686823      
4                                              0.056686      
...                                                 ...      
2926                                           0.897258      
2927                                           1.332285      
2928                                           1.534630      
2929                                           1.026265      
2930                                           0.366412      

      final_cc_cname_DI_mag2_last_event_DI  nplay_video_last_event_DI_max  \
0                                 0.916841                        17314.0   
1                                 1.032418                         1222.0   
2                                 1.107220                         2674.0   
3                                 0.983606                         1782.0   
4                                 0.922133                         3838.0   
...                                    ...                            ...   
2926                              1.244541                         1940.0   
2927                              1.315417                         2197.0   
2928                              0.913509                          551.0   
2929                              1.236641                         1517.0   
2930                              0.638297                          441.0   

      ndays_act_last_event_DI_max  gender_last_event_DI_var  \
0                            91.0                  0.245394   
1                            88.0                  0.201581   
2                            54.0                  0.183983   
3                            92.0                  0.247619   
4                            64.0                  0.251659   
...                           ...                       ...   
2926                         83.0                  0.221053   
2927                         78.0                  0.212903   
2928                         65.0                  0.247485   
2929                         62.0                  0.176101   
2930                         26.0                  0.333333   

      nevents_cv_last_event_DI  explored_mag2_last_event_DI  \
0                     0.783901                     1.044575   
1                     1.234471                     1.533310   
2                     2.078755                     1.571404   
3                     1.275007                     1.749969   
4                     0.742921                     1.105870   
...                        ...                          ...   
2926                  2.311503                     1.428551   
2927                  1.185601                     1.722193   
2928                  0.550733                     1.290892   
2929                  2.627022                     1.124987   
2930                  1.335337                     1.499978   

      YoB_minus2_YoB_last_event_DI_mean  start_time_DI_last_event_DI_nunique  \
0                          -1970.341463                                 99.0   
1                          -1738.739130                                 23.0   
2                          -1823.545455                                 21.0   
3                          -1717.142857                                 21.0   
4                          -1975.074468                                 56.0   
...                                 ...                                  ...   
2926                       -1819.750000                                 18.0   
2927                       -1733.903226                                 29.0   
2928                       -1703.309859                                 62.0   
2929                  

In [96]:
features_train_new.shape

(8791, 500)

In [97]:
features_test_new.shape

(2931, 500)

In [98]:
features_train_new.to_csv('data/features_train_new.csv', index=False)
features_test_new.to_csv('data/features_test_new.csv', index=False)

In [99]:
features_train_new.index = X_train_OE.index
features_train_new = pd.concat([features_train_new, 
                                X_train_OE], axis=1)

In [100]:
features_train_new

start_time_DI_dive1_start_time_DI_last_event_DI_mean  \
3708                                            1.496845      
11541                                           1.742066      
8394                                            1.742066      
5448                                            1.039031      
1423                                            1.099001      
...                                                  ...      
8446                                            1.663552      
5753                                            0.509046      
11125                                           1.750411      
1126                                            0.489385      
2268                                            0.079767      

       final_cc_cname_DI_mag2_last_event_DI  nplay_video_last_event_DI_max  \
3708                               0.932735                    3807.000000   
11541                              0.914185                     437.896054   
8394                               0.914185                     437.896054   
5448                               1.017377                    1516.000000   
1423                               1.028455                     477.000000   
...                                     ...                            ...   
8446                               1.015037                     583.000000   
5753                               0.917910                    1178.000000   
11125                              0.962771                    4312.000000   
1126                               0.999999                     791.000000   
2268                               0.916841                   17314.000000   

       ndays_act_last_event_DI_max  gender_last_event_DI_var  \
3708                          64.0                  0.260000   
11541                        112.0                  0.239505   
8394                         112.0                  0.239505   
5448                          68.0                  0.247354   
1423                          52.0                  0.257576   
...                            ...                       ...   
8446                          63.0                  0.229550   
5753                          28.0                  0.265152   
11125                        116.0                  0.242347   
1126                          16.0                  0.257353   
2268                          91.0                  0.245394   

       nevents_cv_last_event_DI  explored_mag2_last_event_DI  \
3708                   2.391349                     0.000000   
11541                  0.529356                     1.148502   
8394                   0.529356                     1.148502   
5448                   2.105250                     0.000000   
1423                   0.985245                     0.000000   
...                         ...                          ...   
8446                   0.767339                     1.190462   
5753                   0.799921                     0.999980   
11125                  1.100255                     1.324307   
1126                   1.199044                     0.000000   
2268                   0.783901                     1.044575   

       YoB_minus2_YoB_last_event_DI_mean  start_time_DI_last_event_DI_nunique  \
3708                        -1762.884615                                 26.0   
11541                       -1710.672414                                 93.0   
8394                        -1710.672414                                 93.0   
5448                        -1792.750000                                 22.0   
1423                        -1761.757576                                 31.0   
...                                  ...                                  ...   
8446                        -1710.266667                                 59.0   
5753                        -1913.500000                                 10.0   
11125                       -1676.653061      

In [101]:
features_test_new.index = X_test_OE.index
features_test_new = pd.concat([features_test_new, 
                                X_test_OE], axis=1)

In [102]:
features_test_new

start_time_DI_dive1_start_time_DI_last_event_DI_mean  \
10723                                           0.079767      
7157                                            1.288930      
6879                                            1.287020      
3463                                            1.686823      
1319                                            0.056686      
...                                                  ...      
1616                                            0.897258      
11310                                           1.332285      
6249                                            1.534630      
4551                                            1.026265      
5117                                            0.366412      

       final_cc_cname_DI_mag2_last_event_DI  nplay_video_last_event_DI_max  \
10723                              0.916841                        17314.0   
7157                               1.032418                         1222.0   
6879                               1.107220                         2674.0   
3463                               0.983606                         1782.0   
1319                               0.922133                         3838.0   
...                                     ...                            ...   
1616                               1.244541                         1940.0   
11310                              1.315417                         2197.0   
6249                               0.913509                          551.0   
4551                               1.236641                         1517.0   
5117                               0.638297                          441.0   

       ndays_act_last_event_DI_max  gender_last_event_DI_var  \
10723                         91.0                  0.245394   
7157                          88.0                  0.201581   
6879                          54.0                  0.183983   
3463                          92.0                  0.247619   
1319                          64.0                  0.251659   
...                            ...                       ...   
1616                          83.0                  0.221053   
11310                         78.0                  0.212903   
6249                          65.0                  0.247485   
4551                          62.0                  0.176101   
5117                          26.0                  0.333333   

       nevents_cv_last_event_DI  explored_mag2_last_event_DI  \
10723                  0.783901                     1.044575   
7157                   1.234471                     1.533310   
6879                   2.078755                     1.571404   
3463                   1.275007                     1.749969   
1319                   0.742921                     1.105870   
...                         ...                          ...   
1616                   2.311503                     1.428551   
11310                  1.185601                     1.722193   
6249                   0.550733                     1.290892   
4551                   2.627022                     1.124987   
5117                   1.335337                     1.499978   

       YoB_minus2_YoB_last_event_DI_mean  start_time_DI_last_event_DI_nunique  \
10723                       -1970.341463                                 99.0   
7157                        -1738.739130                                 23.0   
6879                        -1823.545455                                 21.0   
3463                        -1717.142857                                 21.0   
1319                        -1975.074468                                 56.0   
...                                  ...                                  ...   
1616                        -1819.750000                                 18.0   
11310                       -1733.903226                                 29.0   
6249                        -1703.309859      

In [103]:
features_train_new.to_csv('data/X_train.csv', index=False)
features_test_new.to_csv('data/X_test.csv', index=False)

In [104]:
y_train.to_csv('data/y_train.csv', index=False)
y_test.to_csv('data/y_test.csv', index=False)